## Connect

In [2]:
import os, sys
sys.path.append(os.path.join('..', 'osiris'))
sys.path.append(os.path.join('..', 'ext'))
import osiris_global
osiris_global.set_runtime_env(interactive_nb=True)

In [3]:
from data.tables import events, actors, mentions, query
eve = mentions('2022-01-01', maxrows=30000)
eve

Fetch event mentions table data:   0%|          | 0/3 [00:00<?, ?batch/s]

,ID,EventID,MentionDate,MentionType,Source,Confidence,Tone
0,https://menafn.com/1104084008/JK-Presenting-Ne...,1040948256,2022-04-24,1,menafn.com,90,1.085142
1,https://www.iol.co.za/mercury/news/police-shoo...,1021543133,2022-01-02,1,iol.co.za,90,-6.666667
2,https://lufkindailynews.com/anpa/us/article_6b...,1040889389,2022-04-24,1,lufkindailynews.com,90,-5.767138
3,https://www.news-gazette.com/opinion/columns/j...,1021536869,2022-01-02,1,news-gazette.com,90,0.769231
4,https://www.strategic-culture.org/news/2022/01...,1021569241,2022-01-02,1,strategic-culture.org,90,-0.730391
...,...,...,...,...,...,...,...
9995,https://www.antaranews.com/berita/2641197/kelu...,1023051780,2022-01-13,1,antaranews.com,80,-4.976303
9996,https://www.nbcwashington.com/news/local/17-ye...,1022993821,2022-01-13,1,nbcwashington.com,80,-2.040816
9997,https://www.europesun.com/news/272166751/right...,1023096941,2022-01-13,1,europesun.com,80,-8.214850
9998,https://www.rtbf.be/auvio/detail_inondations-6...,1023074887,2022-01-13,1,rtbf.be,80,-0.961538


In [3]:
loc = query("SELECT ev.ID, ev.CAMEOCode, ev.Geo_Lat, ev.Geo_Long FROM osiris-347701.gdelt_snapshots.events_actions_20220000_20220421 ev LIMIT 50000")
loc

Fetch query data:   0%|          | 0/5 [00:00<?, ?batch/s]

,ID,CAMEOCode,Geo_Lat,Geo_Long
0,1024310578,0241,37.00000,127.5000
1,1038525737,1056,32.00000,53.0000
2,1021492947,152,23.83640,91.2750
3,1036062520,063,50.92160,34.8003
4,1033389645,101,30.44000,72.3543
...,...,...,...,...
9995,1037426911,1822,46.00000,25.0000
9996,1024937908,1822,-17.78330,25.1667
9997,1036537258,1822,46.84890,35.3653
9998,1031425168,1822,3.71810,98.2202


In [4]:
print(eve)

                                                     ID     EventID  \
0     https://menafn.com/1104084008/JK-Presenting-Ne...  1040948256   
1     https://www.iol.co.za/mercury/news/police-shoo...  1021543133   
2     https://lufkindailynews.com/anpa/us/article_6b...  1040889389   
3     https://www.news-gazette.com/opinion/columns/j...  1021536869   
4     https://www.strategic-culture.org/news/2022/01...  1021569241   
...                                                 ...         ...   
9995  https://www.antaranews.com/berita/2641197/kelu...  1023051780   
9996  https://www.nbcwashington.com/news/local/17-ye...  1022993821   
9997  https://www.europesun.com/news/272166751/right...  1023096941   
9998  https://www.rtbf.be/auvio/detail_inondations-6...  1023074887   
9999  https://rancherita.com.mx/noticias/detalles/11...  1022992712   

     MentionDate  MentionType                 Source  Confidence      Tone  
0     2022-04-24            1             menafn.com          90  1.08

In [7]:
import folium
map_osm = folium.Map(
    location=[48., 31.], 
    tiles="cartodbpositron",
    zoom_start=
)
map_osm

In [3]:
import os, sys
from graphistry import graphistry
#graphistry.register(api=3, username=os.environ['GRAPHISTRY_USER'], password=os.environ['GRAPHISTRY_PASS'], protocol='https', server='hub.graphistry.com')

In [4]:
g = graphistry.tigergraph(
    protocol='https', server='osiris-dev2.i.tgcloud.io',
    user=os.environ['OSIRIS_GRAPH_SERVER_USER'], pwd=os.environ['OSIRIS_GRAPH_SERVER_PASS'], 
    db='GDELT_Events'
)

No authorization token specified for REST++ requests.


## Query and Visualize

In [5]:
g3 = g.gsql("""
  INTERPRET QUERY () FOR GRAPH GDELT_Events { 
   ListAccum<EDGE> @edgeList;
  actors = 
    select s from Actor:s-(Acted>:a)-Event:t
    where t.event_date > "2022-01-01"
    accum s.@edgeList += a
    limit 100;
  print actors[actors.@edgeList];
  }
    """, 
  {'nodes':'actors', 'edges': 'actors.attributes.@edgeList'}
)       
g3.plot()

#g3.plot()

Edges: [].


AttributeError: 'NoneType' object has no attribute 'bind'